In [1]:
import pandas as pd
import numpy as np
import itertools, re
from nltk.stem import StemmerI
from nltk.metrics import edit_distance

In [2]:
class IndianNameStemmer(StemmerI):
    def stem(self, token):
        newtup=list()
        for i in token:
            i = re.sub(r'\([^)]*\)', '', i).strip()
            i = re.sub(r'\[[^)]*\]', '', i).strip()
            i = re.sub(r'(^\w{2,3}\. ?)', r'', i)
            i = i.split('.')[0]
            i = i.split('-')[0]
            i = re.sub(r'[^a-zA-Z]', '', i)
            i = re.sub(r'(\w)\1+',r'\1', i)
            i = i[:-2] if i.endswith(('bi', 'ji', 'so')) else i            
            i = i[:-3] if i.endswith(('bai', 'ben', 'beg', 'tai', 'sab', 'rao', 'rav', 'kha', 'lal', 'dev', 'deo')) else i
            i = i[:-4] if i.endswith(('bhai', 'bhau', 'sing', 'devi')) else i
            i = i[:-5] if i.endswith(('saheb', 'singh' , 'shing', 'kumar')) else i
            for r in (("tha", "ta"), ("krish", "krush"), ("gi", "ji"), ("bh", "b"),  ("yu", "u"), ("gh", "g"),  
                      ("bh","b"), ("ph","f"),  ("gaw", "gaon"), ("ksh", "x"), 
                      ("dh", "d"), ("sh", "s") ):
                # ("g","j"), ("o", "u"), ("i", "e"), ("m", "n"),  ("y", "i"), ("v", "w"), ("z","j"),, ("a","")
                i = i.replace(*r)

            newtup.append(''.join(i for i, _ in itertools.groupby(i)))
        return tuple(newtup)


In [3]:
s = IndianNameStemmer()
s.stem([ 'abbar', 'bhimrav'])


('abar', 'bim')

In [4]:
df=pd.read_excel('correct.xlsx')
df.columns=['wrong', 'correct']

In [5]:
df['updated']=df['wrong'].astype(str).apply(lambda x: x.split('/')).apply(s.stem)
df['updated1']=df['correct'].astype(str).apply(lambda x: x.split('/')).apply(s.stem)

In [6]:
mylist=list()
mylist1=list()
for i, rows in df.iterrows():
    mylist.append(edit_distance(rows['updated'][0], rows['updated1'][0] , substitution_cost=1, transpositions=True))
    mylist1.append(edit_distance(str(rows['wrong']), str(rows['correct']), substitution_cost=1, transpositions=True))


In [7]:
df['edu1'] = mylist
df['wc'] = mylist1

In [8]:
#pd.concat([df.edu1.value_counts(), df.wc.value_counts()], axis=1)

In [10]:
df['tally']=df.wrong.str[:1] == df.correct.str[:1]

In [11]:
df['diff'] = df['wc'] - df['edu1']

In [23]:
ndf=df[~((df['edu1'] > 3) & (df['wc'] > 5) & (df['tally'] == 0) & (df['diff'] < 4) )]

In [24]:
ndf=ndf[~( ((ndf['edu1'] > 3)  | (ndf['wc'] > 3) ) &  (df['tally'] == 0)  )]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [25]:
#ndf=ndf[np.logical_and((ndf['edu1'] > 3) , (ndf['wc'] > 5))]

In [26]:
#ndf=ndf[np.logical_and(np.logical_and((ndf['edu1'] > 3) , (ndf['wc'] > 5)),  (ndf['tally'] == 0 ))]

In [31]:
#ndf['ulen'] = ndf['updated'].str[0].apply(len)
#ndf = ndf.sort_values('ulen')

In [32]:
ndf.to_excel('to_study1.xlsx')

In [22]:
#break here

SyntaxError: invalid syntax (<ipython-input-22-3ef470a3d0aa>, line 1)

# compare names from 2 dataframes

In [ ]:
from io import StringIO

myst="""shantanu prabhakar oka, 905034 , 19:44   
sammeer annashaaheb goankar, 905094  , 19:33
paravatibai vittal shelke,  905154 ,   21:56
"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [ ]:
myst="""shantanu34 prabakar oka, 905034 , 19:44   
sammir anashaheb goankar, 905094  , 19:33
parwati vithal bholke,  905154 ,   21:56
 anashaheb samir gavkar, 905094  , 19:33

"""
u_cols=['name', 'my_index', 'current_tm']

myf = StringIO(myst)
import pandas as pd
df1 = pd.read_csv(StringIO(myst), sep=',', names = u_cols)

In [ ]:
df['updated']=df['name'].astype(str).apply(lambda x: x.split()).apply(s.stem).apply(sorted).apply(tuple)
df1['updated1']=df1['name'].astype(str).apply(lambda x: x.split()).apply(s.stem).apply(sorted).apply(tuple)

In [ ]:
df1

In [ ]:
df

In [ ]:
df.merge(df1, how='outer', left_on=['updated'], right_on=['updated1'], indicator=True)

In [ ]:
mylist=list()
for i, rows in df.iterrows():
        for i1, rows1 in df1.iterrows():
            #mylist.append(edit_distance(rows['updated'], rows1['updated1'], substitution_cost=2))
            #print (rows['updated'], rows1['updated1'], edit_distance(rows['updated'], rows1['updated1'], substitution_cost=2))
            s1, s2 = rows['updated'], rows1['updated1']  
            final=list()
            for i in s1:
                mylisti=list()
                for k in s2:
                    mylisti.append(edit_distance(i, k, transpositions=True))
                final.append(min(mylisti))
            mylist.append(sum(final))
            
            

In [ ]:
mylist

In [ ]:
fdf = pd.merge(df.assign(key=0), df1.assign(key=0), on='key').drop('key', axis=1)

In [ ]:
fdf['eud'] = mylist

In [ ]:
fdf

In [ ]:
gdf=fdf.groupby(['name_x'])['eud'].min()

In [ ]:
gdf=gdf.reset_index()

In [ ]:
gdf

In [ ]:
gdf.merge(fdf, how='left', left_on=['name_x', 'eud'], right_on=['name_x', 'eud'])